# 05. 오차역전파법

가중치 매개변수의 기울기를 효율적으로 계산하기 위해 **오차역전파법(backpropagation)** 을 이용한다.

## 5.1. 계산 그래프

* 계산 그래프를 사용하면 **국소적 계산** 이 가능하다.
* 중간에 구한 미분 결과를 공유할 수 있어서 다수의 미분을 효율적으로 계산할 수 있다.

## 5.2. 연쇄법칙

* 합성함수의 미분은 합성함수를 구성하는 각 함수의 미분의 곱으로 구할 수 있다.
* 계산 그래프의 역전파는 연쇄법칙의 원리를 따른다.

## 5.3. 역전파

* **덧셈 노드**의 역전파는 입력 신호를 그대로 다음 노드에 출력한다.

* **곱셈 노드**의 역전파는 입력 신호에 순전파 때의 입력값들을 '서로 바꾼 값'을 곱해서 구할 수 있다.
* 곱셈 노드를 구현할 때에는 순전파의 입력 신호를 변수에 저장해둔다.

## 5.4. 단순한 계층 구현하기

* 모든 계층은 forward()와 backward()라는 공통의 메서드(인터페이스)를 갖도록 구현한다.
* forward()는 순전파, backward()는 역전파를 처리한다.

#### 5.4.1. 곱셈 계층

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y                
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x

        return dx, dy

In [2]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

# output
print(price) # 220

220.00000000000003


In [3]:
# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

# 갹 변수에 대한 미분
print(dapple, dapple_num, dtax) # 2.2 110 200

2.2 110.00000000000001 200


#### 5.4.2. 덧셈 계층

In [4]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

#### buy_apple_orange

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


## 5.5. 활성화 함수 계층 구현하기

#### 5.5.1. ReLU 계층

In [6]:
import numpy as np

class Relu:
    def __init__(self):
        self.mask = None  # mask : True/False로 구성된 넘파이 배열 (순전파 입력값이 0 이하면 True, 그 외는 False)

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [7]:
x = np.array([[1.0, -0.5], [2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [ 2.   3. ]]


In [8]:
 # mask : True/False로 구성된 넘파이 배열 (순전파 입력값이 0 이하면 True, 그 외는 False)
mask = (x <= 0)
print(mask)

[[False  True]
 [False False]]


#### 5.5.2. Sigmoid 계층

In [9]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

## 5.6. Affine/Softmax 계층 구현하기

#### Affine 계층

In [10]:
# 데이터가 2개(N=2)라고 가정
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

#### Softmax-with-Loss 계층

In [11]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

## 5.7. 오차역전파법 구현하기

#### 오차역전파법을 이용한 신경망 구현

In [35]:
os.getcwd()

'C:\\Users\\Soyeon\\Desktop\\Study\\DL-Study\\밑시딥\\deep-learning-from-scratch\\ch05'

In [34]:
import sys, os
os.chdir("C:/Users/Soyeon/Desktop/Study/DL-Study/밑시딥/deep-learning-from-scratch/ch05/")
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()  # 순서가 있는 딕셔너리
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

#### 오차역전파법으로 구한 기울기 검증하기

In [36]:
os.getcwd()

'C:\\Users\\Soyeon\\Desktop\\Study\\DL-Study\\밑시딥\\deep-learning-from-scratch\\ch05'

In [37]:
sys.path.append(os.pardir)
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 낸다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:4.818719026104199e-10
b1:2.9595323862316403e-09
W2:5.853701683250891e-09
b2:1.400531036019337e-07


이 결과는 수치 미분과 오차역전파법으로 구한 기울기의 차이가 매우 작다고 말해준다.

#### 오차역전파법을 사용한 학습 구현하기

In [38]:
sys.path.append(os.pardir)
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)


0.07215 0.0732
0.90175 0.9038
0.9232333333333334 0.9237
0.9353666666666667 0.9338
0.9422 0.9382
0.9521333333333334 0.9475
0.95605 0.9507
0.9601833333333334 0.9558
0.9635833333333333 0.9596
0.96605 0.9604
0.97015 0.9628
0.9716833333333333 0.9648
0.97395 0.9654
0.9756 0.9668
0.9758666666666667 0.967
0.9777666666666667 0.9681
0.9787166666666667 0.969
